OBJECTIVE: Obtain nominal TF as function of MX and also for each variation

In [ ]:
from utils.analysis.feyn import model_path
model_dir = model_path.split('/')[-3]
# model_savein = f'plots/feynnet/{model_dir}'

In [ ]:
data = Data('/eos/uscms/store/user/srosenzw/sixb/ntuples/Summer2018UL/maxbtag_4b/JetHT_Data_UL/ntuple.root')
data.spherical_region()
data.train()